# SBUS Demo Notebook

This notebook demonstrates the SBUS workflow using the organized data structure.

**Links:**
- [Source data](https://ichoosr.egnyte.com/navigate/folder/701cce7f-a797-47c6-bcae-7ee2e3242e8c)
- [Confluence documentation](https://ichoosr.atlassian.net/wiki/x/IYBwMQE?atlOrigin=eyJpIjoiMzg2YWQxZmI2NjUyNDg1Y2E2YjMyZTViOGRjOWFmOGEiLCJwIjoiYyJ9)

In [ ]:
# Setup: Load IPython extensions and configure environment
%reload_ext setup

In [ ]:
# Reset capacity before running (important!)
!python ../scripts/reset_capacity.py --all

In [ ]:
import logging
logging.getLogger("igent").setLevel(logging.DEBUG)
# Example: Run workflow with simplified API
from igent.workflows import run_workflow

await run_workflow(
    constellation="p1m1m2c",
    model="openai_gpt5",
    business_line="sbus",
    data_dir="../data/sbus",
    scenario="overlap_only",
    max_items=1,
)

## Unified Workflow API (Recommended)

The new configuration-driven approach supports multiple constellation patterns:
- **p1m1m2c** - Matcher1+Critic → Matcher2 (current default)
- **p1m1c1m2c2** - All 4 agents in single group
- **p1m1_p2m2** - Two matchers, no critics (fast)
- **p1m1c1_p2m2c2** - Two sequential pairs with critics

**Scenarios** (data organization):
- `overlap_only.yaml` - High competition (all registrations in overlapping ZIPs)
- `full_dataset.yaml` - Diverse distribution (41 different ZIP codes)
- `no_battery.yaml` - Product constraint (All Energy Solar missing batteries)

In [ ]:
# Reset capacity before running (important!)
!python ../scripts/reset_capacity.py --all

In [ ]:
# Run workflow with scenario + constellation configuration
from igent.workflows.p1m1m2c import run_workflow_from_scenario

await run_workflow_from_scenario(
    scenario_file="../data/sbus/scenarios/overlap_only.yaml",
    model="zai_glm4_5_air",  # Model: zai_glm4_5_air, zai_glm4_6, openai_gpt4o, openai_gpt5, azure
    constellation="p1m1m2c",  # Constellation: p1m1m2c, p1m1c1m2c2, p1m1_p2m2, p1m1c1_p2m2c2
    max_items=5,
    stream=False
)

## Direct File Path Workflow (Legacy)

For backward compatibility, you can still specify files directly without using scenarios.

In [ ]:
# Direct file path approach (less recommended)
from igent.workflows.p1m1m2c import run_workflow

await run_workflow(
    model="zai_glm4_5_air",
    stream=False,
    business_line="sbus",
    registrations_file="../data/sbus/registrations/overlap_only.json",
    offers_file="../data/sbus/offers/base_offers.json",
    matches_file="../data/sbus/results/matches.json",
    pos_file="../data/sbus/results/pos.json",
    stats_file="../data/sbus/results/stats.csv",
    max_items=5
)

## List Available Scenarios

Use this to see what scenario configurations are available.

In [ ]:
# List all available scenarios
from igent.utils import list_scenarios, load_scenario

scenarios = list_scenarios("../data/sbus/scenarios")
print("Available scenarios:")
for scenario_file in scenarios:
    config = load_scenario(scenario_file)
    print(f"\n  📋 {scenario_file}")
    print(f"     {config['name']}")
    print(f"     {config['description']}")

## Try Different Constellations

Experiment with different agent configurations to compare performance and quality.

## Simplified API with Smart Defaults

The new smart defaults feature eliminates repetitive file paths. Just specify `data_dir` and `scenario`, and the workflow will automatically construct all file paths:

**Before** (verbose):
```python
await run_workflow(
    constellation="p1m1m2c",
    model="zai_glm4_5_air",
    business_line="sbus",
    registrations_file="../data/sbus/registrations/overlap_only.json",
    offers_file="../data/sbus/offers/base_offers.json",
    matches_file="../data/sbus/results/p1m1m2c_matches.json",
    pos_file="../data/sbus/results/p1m1m2c_pos.json",
    stats_file="../data/sbus/results/p1m1m2c_stats.csv",
    max_items=5
)
```

**After** (concise):
```python
await run_workflow(
    constellation="p1m1m2c",
    model="zai_glm4_5_air",
    business_line="sbus",
    data_dir="../data/sbus",
    scenario="overlap_only",
    max_items=5
)
```

In [ ]:
# Example: Run workflow with simplified API
from igent.workflows import run_workflow

await run_workflow(
    constellation="p1m1m2c",
    model="zai_glm4_5_air",
    business_line="sbus",
    data_dir="../data/sbus",
    scenario="overlap_only",
    max_items=5
)

In [ ]:
# Compare different constellations using simplified API
from igent.workflows import run_workflow

constellations = ["p1m1m2c", "p1m1_p2m2", "p1m1c1_p2m2c2"]

for constellation in constellations:
    print(f"\n{'='*60}")
    print(f"Testing constellation: {constellation}")
    print(f"{'='*60}\n")
    
    # Simplified API - just specify data_dir and scenario!
    await run_workflow(
        constellation=constellation,
        model="zai_glm4_5_air",
        business_line="sbus",
        data_dir="../data/sbus",
        scenario="overlap_only",
        max_items=2
    )
    
print("\n✅ Constellation comparison complete!")